In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ls "/content/drive/My Drive/NLP-Project"

Mounted at /content/drive
2019VAERSDATA.csv      2020VAERSVAX.csv       2022VAERSSYMPTOMS.csv  ft_100d_oa_all.tar.gz
2019VAERSSYMPTOMS.csv  2021VAERSDATA.csv      2022VAERSVAX.csv	     gl_100d_oa_all.tar.gz
2019VAERSVAX.csv       2021VAERSSYMPTOMS.csv  2023VAERSDATA.csv
2020VAERSDATA.csv      2021VAERSVAX.csv       2023VAERSSYMPTOMS.csv
2020VAERSSYMPTOMS.csv  2022VAERSDATA.csv      2023VAERSVAX.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import ast
import numpy as np
warnings.filterwarnings("ignore")
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
symptoms_data_actual = pd.read_csv("/content/symptoms_output.csv")
symptoms_data_extracted = pd.read_csv("/content/symptoms_data_extracted.csv")
symptoms_data_actual['Symptoms'] = symptoms_data_actual['Symptoms'].apply(ast.literal_eval)
symptoms_data_extracted['Symptoms'] = symptoms_data_extracted['Symptoms'].apply(ast.literal_eval)

In [ ]:
# Working on only 20 datapoints
symptoms_data_actual_20 = symptoms_data_actual.head(1000)
symptoms_data_extracted_20 = symptoms_data_extracted.head(1000)

In [ ]:
symptoms_data_extracted_20_flatten = set()

for symptom_list in symptoms_data_extracted_20["Symptoms"]:
    for symptoms in symptom_list:
        symptoms_data_extracted_20_flatten.add(symptoms)

# Convert the set to a list if needed
symptoms_data_extracted_20_flatten = list(symptoms_data_extracted_20_flatten)


In [ ]:
symptoms_data_actual_20_flatten = []

for symptom_list in symptoms_data_actual_20["Symptoms"]:
    for symptoms in symptom_list:
        symptoms_data_actual_20_flatten.append(symptoms)



In [ ]:
from collections import Counter
# Count occurrences of each symptom
symptom_counts = Counter(symptoms_data_actual_20_flatten)
most_frequent_actual_symptoms = {symptom: count for symptom, count in symptom_counts.items() if count > 5}

In [ ]:
most_frequent_actual_symptoms

{'Heavy menstrual bleeding': 12,
 'Menstruation irregular': 9,
 'SARS-CoV-2 test negative': 17,
 'Chills': 75,
 'Fatigue': 115,
 'Immunisation': 57,
 'Pain': 117,
 'Diarrhoea': 26,
 'SARS-CoV-2 test': 31,
 'Lymphadenopathy': 49,
 'Rhinorrhoea': 8,
 'Vaccination site pain': 14,
 'Rash': 23,
 'Pyrexia': 101,
 'Headache': 128,
 'Illness': 18,
 'Vomiting': 44,
 'Abdominal pain upper': 12,
 'Hypertension': 15,
 'Heart rate increased': 28,
 'Palpitations': 28,
 'Lymph node pain': 15,
 'Myalgia': 29,
 'Mobility decreased': 11,
 'Paraesthesia': 19,
 'Hypoaesthesia': 29,
 'Neck pain': 11,
 'Feeling abnormal': 27,
 'Insomnia': 13,
 'Dizziness': 70,
 'Axillary pain': 15,
 'Urticaria': 28,
 'Feeling hot': 7,
 'Influenza like illness': 12,
 'Body temperature': 11,
 'Nausea': 74,
 'Erythema': 12,
 'Malaise': 28,
 'Blood test': 35,
 'Pain in extremity': 85,
 'Tremor': 7,
 'Feeling cold': 7,
 'X-ray': 9,
 'Peripheral swelling': 14,
 'SARS-CoV-2 antibody test': 7,
 'Cough': 55,
 'Anosmia': 6,
 'Swellin

In [ ]:
standard_symptoms = list(symptom_counts.keys())
most_common_standard_symptoms = list(most_frequent_actual_symptoms.keys())

In [ ]:
# Remove duplicates in-place using set
unique_symptoms_set = set(symptoms_data_extracted_20_flatten)
symptoms_data_extracted_20_flatten.clear()
symptoms_data_extracted_20_flatten.extend(unique_symptoms_set)

In [ ]:
import tarfile

# Path to the tar.gz file
file_path = '/content/drive/My Drive/NLP-Project/gl_100d_oa_all.tar.gz'

# Extract the contents of the tar.gz file
with tarfile.open(file_path, 'r:gz') as tar:
    tar.extractall('/content/')

In [ ]:
# def load_glove_embeddings(file_path):
#     embeddings_index = {}
#     with open(file_path, encoding="utf-8") as f:
#         for line in f:
#             try:
#                 # Remove non-breaking spaces
#                 line = line.replace('\u2028', ' ')

#                 values = line.split()
#                 word = values[0]
#                 coefs = np.asarray(values[1:], dtype='float32')
#                 embeddings_index[word] = coefs
#             except ValueError as e:
#                 # print(f"Error processing line: {line}")
#                 print(f"Error message: {e}")
#     return embeddings_index

# glove_file_path = '/content/GL_100/converted_to_word2vec_100.txt'
# glove_embeddings = load_glove_embeddings(glove_file_path)


Error message: could not convert string to float: 'depressant'
Error message: could not convert string to float: 'degeneration'
Error message: could not convert string to float: 'degenerative_diseases'
Error message: could not convert string to float: 'transmitter'
Error message: could not convert string to float: 'degenerative'
Error message: could not convert string to float: 'macological'


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

# Function to embed text
def embed_text(text, embeddings):
    tokens = word_tokenize(text.lower())
    embedded_text = [embeddings[token] for token in tokens if token in embeddings]
    return np.mean(embedded_text, axis=0) if embedded_text else None

# Calculate similarities and store the results in a dictionary
symptom_linking_dict = {}

for raw_symptom in symptoms_data_extracted_20_flatten:
    embedded_raw_symptom = embed_text(raw_symptom, glove_embeddings)

    max_similarity = 0.0
    linked_standard_symptom = None

    if embedded_raw_symptom is not None:  # Check if the embedding is not None
        for standard_symptom in standard_symptoms:
            embedded_standard_symptom = embed_text(standard_symptom, glove_embeddings)

            if embedded_standard_symptom is not None:  # Check if the embedding is not None
                similarity_score = cosine_similarity([embedded_raw_symptom], [embedded_standard_symptom])[0][0]

                if similarity_score > max_similarity:
                    max_similarity = similarity_score
                    linked_standard_symptom = standard_symptom

    symptom_linking_dict[raw_symptom] = linked_standard_symptom


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

# Function to embed text
def embed_text(text, embeddings):
    tokens = word_tokenize(text.lower())
    embedded_text = [embeddings[token] for token in tokens if token in embeddings]
    return np.mean(embedded_text, axis=0) if embedded_text else None

# Calculate similarities and store the results in a dictionary
most_frequent_symptom_linking_dict = {}

for raw_symptom in symptoms_data_extracted_20_flatten:
    embedded_raw_symptom = embed_text(raw_symptom, glove_embeddings)

    max_similarity = 0.0
    linked_standard_symptom = None

    if embedded_raw_symptom is not None:  # Check if the embedding is not None
        for standard_symptom in most_common_standard_symptoms:
            embedded_standard_symptom = embed_text(standard_symptom, glove_embeddings)

            if embedded_standard_symptom is not None:  # Check if the embedding is not None
                similarity_score = cosine_similarity([embedded_raw_symptom], [embedded_standard_symptom])[0][0]

                if similarity_score > max_similarity:
                    max_similarity = similarity_score
                    linked_standard_symptom = standard_symptom

        most_frequent_symptom_linking_dict[raw_symptom] = linked_standard_symptom


In [ ]:
import pandas as pd

# Create an empty DataFrame
columns = ['VAERS_ID', 'Symptoms', 'Standard_Symptoms']
Symptoms_linked_entities_output = pd.DataFrame(columns=columns)

# Dictionary to store symptoms and standard symptoms lists for each VAERS_ID
symptom_dict = {}

# Iterate over rows in symptoms_data_extracted_20 and append to the DataFrame
for index, row in symptoms_data_extracted_20.iterrows():
    vaers_id = row['VAERS_ID']
    symptoms_list = row['Symptoms']

    for raw_symptom in symptoms_list:
        linked_standard_symptom = symptom_linking_dict.get(raw_symptom)

        if vaers_id not in symptom_dict:
            symptom_dict[vaers_id] = {'Symptoms': [], 'Standard_Symptoms': set()}

        symptom_dict[vaers_id]['Symptoms'].append(raw_symptom)

        # Check if the standard symptom is not seen before
        if linked_standard_symptom and linked_standard_symptom not in symptom_dict[vaers_id]['Standard_Symptoms']:
            symptom_dict[vaers_id]['Standard_Symptoms'].add(linked_standard_symptom)

# Populate the DataFrame with aggregated lists
for vaers_id, symptoms_info in symptom_dict.items():
    Symptoms_linked_entities_output = Symptoms_linked_entities_output.append({
        'VAERS_ID': vaers_id,
        'Symptoms': symptoms_info['Symptoms'],
        'Standard_Symptoms': list(symptoms_info['Standard_Symptoms'])
    }, ignore_index=True)

In [ ]:
# Add a new column for Most Frequent Standard Symptoms to the existing DataFrame
Symptoms_linked_entities_output['Most_Frequent_Standard_Symptoms'] = None

# Set to keep track of seen most frequent standard symptoms
seen_most_frequent_standard_symptoms = set()

# Iterate over rows in the DataFrame and update the new column
for index, row in Symptoms_linked_entities_output.iterrows():
    vaers_id = row['VAERS_ID']
    symptoms_list = row['Symptoms']  # Replace 'Symptoms' with the actual column name

    most_frequent_standard_symptoms = []

    for symptom in symptoms_list:
        most_frequent_standard_symptom = most_frequent_symptom_linking_dict.get(symptom)

        # Check if the most frequent standard symptom is not seen before
        if most_frequent_standard_symptom and most_frequent_standard_symptom not in seen_most_frequent_standard_symptoms:
            most_frequent_standard_symptoms.append(most_frequent_standard_symptom)
            seen_most_frequent_standard_symptoms.add(most_frequent_standard_symptom)

    Symptoms_linked_entities_output.at[index, 'Most_Frequent_Standard_Symptoms'] = most_frequent_standard_symptoms


In [ ]:
actual_symptoms_column = symptoms_data_actual_20['Symptoms']
Symptoms_linked_entities_output.insert(1, 'Actual Symptoms', actual_symptoms_column)
print(Symptoms_linked_entities_output)


    VAERS_ID                                    Actual Symptoms  \
0    1996876  [Dysmenorrhoea, Heavy menstrual bleeding, Mens...   
1    1996877  [Heavy menstrual bleeding, Oligomenorrhoea, SA...   
2    1996878              [Chills, Fatigue, Immunisation, Pain]   
3    1996879                       [Diarrhoea, SARS-CoV-2 test]   
4    1996880  [Fatigue, Immunisation, Lymphadenopathy, Rhino...   
..       ...                                                ...   
856  2000245  [Cardiac stress test, Echocardiogram, Photopsi...   
857  2000246  [COVID-19, Exposure to SARS-CoV-2, Malaise, SA...   
858  2000251          [Pain, Pain in extremity, Sleep disorder]   
859  2000257                             [Rash, Rash vesicular]   
860  2000265                     [Arthralgia, Myalgia, Pyrexia]   

                                              Symptoms  \
0    [irregular menstrual cycle, bad cramping, mens...   
1    [heavy menstrual bleeding, heavy menstrual ble...   
2    [tired, achy, ch

In [ ]:
Symptoms_linked_entities_output.to_csv("symptoms_linked_entities_output.csv", index = False)

In [ ]:
def calculate_precision_recall_f1(target_symptoms, predicted_symptoms):
    true_positives = len(set(target_symptoms).intersection(predicted_symptoms))
    false_positives = len(predicted_symptoms) - true_positives
    false_negatives = len(target_symptoms) - true_positives

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# Create a DataFrame to store individual precision, recall, and F1-score for each VAERS ID
evaluation_results = pd.DataFrame(columns=['VAERS_ID', 'Precision', 'Recall', 'F1'])

# Iterate over unique VAERS IDs
unique_vaers_ids = Symptoms_linked_entities_output['VAERS_ID'].unique()
for vaers_id in unique_vaers_ids:
    # Extract the target and predicted symptoms for the current VAERS ID
    target_symptoms = set(symptoms_data_actual_20[symptoms_data_actual_20['VAERS_ID'] == vaers_id]['Symptoms'].explode())
    predicted_symptoms = set(Symptoms_linked_entities_output[Symptoms_linked_entities_output['VAERS_ID'] == vaers_id]['Standard_Symptoms'].explode())

    # Calculate metrics and append the results to the DataFrame
    precision, recall, f1 = calculate_precision_recall_f1(target_symptoms, predicted_symptoms)
    evaluation_results = evaluation_results.append({
        'VAERS_ID': vaers_id,
        'Precision': precision,
        'Recall': recall,
        'F1': f1
    }, ignore_index=True)

# Calculate average precision, recall, and F1-score across all VAERS IDs
average_precision = evaluation_results['Precision'].mean()
average_recall = evaluation_results['Recall'].mean()
average_f1 = evaluation_results['F1'].mean()

print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-score: {average_f1}")

Average Precision: 0.37402883647794233
Average Recall: 0.3574799064039877
Average F1-score: 0.3392060323725159


In [ ]:
evaluation_results.to_csv("evaluation_results.csv", index = False)

In [ ]:
# Create a DataFrame to store individual precision, recall, and F1-score for each VAERS ID
evaluation_results_most_frequent = pd.DataFrame(columns=['VAERS_ID', 'Precision', 'Recall', 'F1'])

# Iterate over unique VAERS IDs
unique_vaers_ids_most_frequent = Symptoms_linked_entities_output['VAERS_ID'].unique()
for vaers_id in unique_vaers_ids_most_frequent:
    # Extract the target and predicted symptoms for the current VAERS ID
    target_symptoms_most_frequent = set(symptoms_data_actual_20[symptoms_data_actual_20['VAERS_ID'] == vaers_id]['Symptoms'].explode())
    predicted_symptoms_most_frequent = set(Symptoms_linked_entities_output[Symptoms_linked_entities_output['VAERS_ID'] == vaers_id]['Most_Frequent_Standard_Symptoms'].explode())

    # Calculate metrics and append the results to the DataFrame
    precision_most_frequent, recall_most_frequent, f1_most_frequent = calculate_precision_recall_f1(target_symptoms_most_frequent, predicted_symptoms_most_frequent)
    evaluation_results_most_frequent = evaluation_results_most_frequent.append({
        'VAERS_ID': vaers_id,
        'Precision': precision_most_frequent,
        'Recall': recall_most_frequent,
        'F1': f1_most_frequent
    }, ignore_index=True)

# Calculate average precision, recall, and F1-score across all VAERS IDs
average_precision_most_frequent = evaluation_results_most_frequent['Precision'].mean()
average_recall_most_frequent = evaluation_results_most_frequent['Recall'].mean()
average_f1_most_frequent = evaluation_results_most_frequent['F1'].mean()

print(f"Average Precision (Most Frequent): {average_precision_most_frequent}")
print(f"Average Recall (Most Frequent): {average_recall_most_frequent}")
print(f"Average F1-score (Most Frequent): {average_f1_most_frequent}")


Average Precision (Most Frequent): 0.020376822815847205
Average Recall (Most Frequent): 0.011862033037710235
Average F1-score (Most Frequent): 0.012565928071154552


In [ ]:
evaluation_results_most_frequent.to_csv("evaluation_results_most_frequent.csv", index = False)

In [ ]:
gptoutput = pd.read_csv("/content/finaloutput.csv")

In [ ]:
gptoutput.head(10)

,Unnamed: 0.1,VAERS_ID,Symptoms_actual,Symptoms_extracted,Unnamed: 0,Generated_Response
0,0,1996876,"['Dysmenorrhoea', 'Heavy menstrual bleeding', ...","['bad cramping', 'dysmenorrhoea', 'menstruatio...",0,"['Dysmenorrhoea', 'Dysmenorrhoea', 'Menstruati..."
1,1,1996877,"['Heavy menstrual bleeding', 'Oligomenorrhoea'...",['heavy menstrual bleeding'],1,"['Heavy menstrual bleeding', 'Oligomenorrhoea']"
2,2,1996878,"['Chills', 'Fatigue', 'Immunisation', 'Pain']","['side effects', 'achy', 'fatigue', 'pain', 't...",2,"['Fatigue', 'Pain', 'Chills']"
3,3,1996879,"['Diarrhoea', 'SARS-CoV-2 test']",['activated immune system viral complications'...,3,"['Diarrhoea', 'Thyroid disorder', 'Mononucleos..."
4,4,1996880,"['Fatigue', 'Immunisation', 'Lymphadenopathy',...","['prediabetes', 'nose running', 'sore arm jab ...",4,"['Fatigue', 'Rhinorrhoea', 'Lymphadenopathy', ..."
5,5,1996881,"['Biopsy', 'Biopsy skin', 'Drug eruption', 'Ra...","['lichenoid drug eruption', 'depression', 'tor...",5,"['Drug eruption', 'Rash']"
6,6,1996882,['Pyrexia'],"['fever', 'patient pain', 'patient s pain', 'p...",6,"['Pyrexia', 'Pain']"
7,7,1996883,"['Fatigue', 'Headache', 'Vaccination site pain']","['fatigue', 'vaccination site pain', 'headache']",7,"['Fatigue', 'Vaccination site pain', 'Headache']"
8,8,1996884,"['Illness', 'Vomiting']",['vomiting'],8,['Vomiting']
9,9,1996885,['Abdominal pain upper'],"['abdominal pain upper nonserious onset', 's...",9,"['Abdominal pain upper', 'Stomach issues']"


In [ ]:
gptoutput.columns

Index(['Unnamed: 0.1', 'VAERS_ID', 'Symptoms_actual', 'Symptoms_extracted',
       'Unnamed: 0', 'Generated_Response'],
      dtype='object')

In [ ]:
def calculate_metrics(target_symptoms, predicted_symptoms):
    true_positives = len(set(target_symptoms).intersection(predicted_symptoms))
    false_positives = len(predicted_symptoms) - true_positives
    false_negatives = len(target_symptoms) - true_positives

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# Create a DataFrame to store individual precision, recall, and F1-score for each record
evaluation_results_gpt = pd.DataFrame(columns=['Precision', 'Recall', 'F1'])

# Iterate over each record in the DataFrame
for index, row in gptoutput.iterrows():
    # Extract the target and predicted symptoms for the current record
    target_symptoms_gpt = set(row['Symptoms_actual'])
    predicted_symptoms_gpt = set(row['Generated_Response'])

    # Calculate metrics and append the results to the DataFrame
    precision_gpt, recall_gpt, f1_gpt = calculate_metrics(target_symptoms_gpt, predicted_symptoms_gpt)
    evaluation_results_gpt = evaluation_results_gpt.append({
        'Precision': precision_gpt,
        'Recall': recall_gpt,
        'F1': f1_gpt
    }, ignore_index=True)

# Calculate average precision, recall, and F1-score across all records
average_precision_gpt = evaluation_results_gpt['Precision'].mean()
average_recall_gpt = evaluation_results_gpt['Recall'].mean()
average_f1_gpt = evaluation_results_gpt['F1'].mean()

print(f"Average Precision: {average_precision_gpt}")
print(f"Average Recall: {average_recall_gpt}")
print(f"Average F1-score: {average_f1_gpt}")

Average Precision: 0.9322535039566947
Average Recall: 0.8604722357983066
Average F1-score: 0.8836887767606412


In [ ]:
evaluation_results_gpt.to_csv("evaluation_results_gpt.csv", index = False)